## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

reverse engineering this for out own model

In [4]:
!git clone https://github.com/leeex1/Quillan-v4.2-repo.git

Cloning into 'Quillan-v4.2-repo'...
remote: Enumerating objects: 2208, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 2208 (delta 186), reused 39 (delta 39), pack-reused 1939 (from 2)
Receiving objects: 100% (2208/2208), 42.12 MiB | 27.56 MiB/s, done.
Resolving deltas: 100% (1325/1325), done.


In [30]:
import os
import json

# --- Paths ---
jsonl_file_path = "/content/Quillan-v4.2-repo/Quillan-v4.2-model/Quillan_finetune_full_dataset.jsonl"
quillan_files_dir = "/content/Quillan-v4.2-repo/Quillan files"

# --- Function to recursively extract text/content from JSON objects (for .jsonl lines) ---
def find_text_in_json(data):
    texts = []
    if isinstance(data, str):
        texts.append(data)
    elif isinstance(data, dict):
        for key, value in data.items():
            texts.extend(find_text_in_json(value))
    elif isinstance(data, list):
        for item in data:
            texts.extend(find_text_in_json(item))
    return texts

# --- Load main dataset file (.jsonl) ---
main_samples = []
if os.path.exists(jsonl_file_path):
    with open(jsonl_file_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data = json.loads(line.strip())
                # Recursively grab all string content
                main_samples.extend(find_text_in_json(data))
            except Exception:
                continue
else:
    print(f"❌ Could not find {jsonl_file_path}")

print(f"Loaded {len(main_samples):,} samples from Quillan_finetune_full_dataset.jsonl")

# --- Load all text from every file in the Quillan directory ---
quillan_file_texts = []
for filename in os.listdir(quillan_files_dir):
    file_path = os.path.join(quillan_files_dir, filename)
    try:
        # Only use non-binary and non-image/text files (skip .gitkeep, images, etc.)
        if not filename.lower().endswith(('.png', '.jpg', 'jpeg', '.bmp', '.gif')):
            with open(file_path, "r", encoding="utf-8") as fp:
                text = fp.read()
                # Only add meaningful content (skip tiny/empty)
                if len(text.strip()) > 10:
                    quillan_file_texts.append(text)
    except Exception as e:
        print(f"Skipped {filename}: {e}")

print(f"Loaded {len(quillan_file_texts)} files from Quillan folder.")

# --- FULL MERGED CORPUS ---
# Combine everything as a single string (or list of samples if batching line-by-line)
full_corpus = "\n\n".join(main_samples + quillan_file_texts)

print("Merged corpus length (characters):", len(full_corpus))

# --- Tokenizer and batching as before ---
# At this point, run all splitting, tokenization, vocab building, batching, etc. on `full_corpus`
# Example:
chars = sorted(list(set(full_corpus)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
import torch
data = torch.tensor(encode(full_corpus), dtype=torch.long)
print("Tokens shape:", data.shape)

# Continue train/val split, batching, training, and model code as before...
# train_data, val_data = torch.split(data ...
# etc.


Loaded 432 samples from Quillan_finetune_full_dataset.jsonl
Loaded 57 files from Quillan folder.
Merged corpus length (characters): 8972889
Tokens shape: torch.Size([8972889])


In [33]:
# Inspect the combined text content
# This variable was created in the previous cell by reading multiple files.
# Print the first 2500 characters to inspect it
print(f"Full corpus length: {len(full_corpus)}")

Full corpus length: 8972889


In [34]:
# Train/validation split (90% train, 10% val)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print("Train tokens:", train_data.shape)
print("Val tokens:", val_data.shape)


Train tokens: torch.Size([8075600])
Val tokens: torch.Size([897289])


In [35]:
import torch

batch_size = 32         # sequences per batch
block_size = 256        # max context length (matches model config)

def get_batch(split):
    d = train_data if split == "train" else val_data
    ix = torch.randint(len(d) - block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size] for i in ix])
    y = torch.stack([d[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("Input shape:", xb.shape)
print("Target shape:", yb.shape)


Input shape: torch.Size([32, 256])
Target shape: torch.Size([32, 256])


In [39]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ §«¬­±²·»Ó×áäæéíïóôöüčŌōʹΓΔΘΞΣΦΨΩαβγδζηθικλμνξοπρςστφχψωόϕϵ     ​‍‐‑–—‘’“”†•…‪‬⁡₁₂€ℎℰ↑→↓↔∈∏∑−∙√∣∥∫≈≠≥≲≻⊗⋯⏰⏱─□◇○●☠♀♂♠♣♥♦♾♿⚖⚙⚠⚡✅✍✓✗✨❌❤➔➡⟩⨂⭐〉【】扮演️﻿𝐸𝑁𝑂𝑆𝑎𝑐𝑑𝑒𝑓𝑔𝑖𝑗𝑘𝑙𝑚𝑛𝑜𝑝𝑟𝑠𝑡𝑢𝑣𝑤𝑥𝑦𝑧𝖫𝖬𝛼𝛽𝛾𝛿𝜁𝜂𝜃𝜅𝜆𝜇𝜖𝜙🌀🌈🌉🌊🌌🌍🌐🌙🌟🌪🌱🌳🎉🎨🎪🎭🎮🎯🎵🏃🏆🏎🏗🏛🏹👁👋👑👤👥💀💎💖💝💡💪💫💬💭💯💻💾📈📊📋📌📖📘📚📜📝📤🔀🔁🔄🔍🔑🔒🔗🔚🔢🔥🔧🔬🔮🔹🕰🕳🕸🗣🗺😊🚀🚦🚨🚪🚫🚶🛠🛡🛤🤔🤖🤗🤝🦉🧠🧩🧬🧭🧮🧶
374


In [40]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hi there, how are you today?"))
print(decode(encode("hi there, how are you today?")))

[74, 75, 2, 86, 74, 71, 84, 71, 14, 2, 74, 81, 89, 2, 67, 84, 71, 2, 91, 81, 87, 2, 86, 81, 70, 67, 91, 33]
hi there, how are you today?


In [41]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([46687]) torch.int64
tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17, 71, 80, 88,  2, 82, 91, 86,
        74, 81, 80, 21,  1,  4,  4,  4,  1, 51, 87, 75, 78, 78, 67, 80,  2, 37,
        49, 48, 53, 37, 43, 49, 55, 53, 48, 39, 53, 53,  2, 47, 55, 46, 54, 43,
        47, 49, 38, 35, 46,  2, 40, 55, 53, 43, 49, 48,  2, 39, 48, 41, 43, 48,
        39,  2, 88, 22, 16, 20, 16, 19,  1, 31, 31, 31, 31, 31, 31, 31, 31, 31,
        31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
        31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
        31, 31, 31, 31,  1, 47, 87, 78, 86, 75, 79, 81, 70, 67, 78,  2, 72, 87,
        85, 75, 81, 80,  2, 67, 78, 75, 73, 80, 71, 70,  2, 86, 81,  2, 70, 91,
        80, 67, 79, 75, 69,  2, 69, 81, 80, 85, 69, 75, 81, 87, 85, 80, 71, 85,
        85,  2, 86, 71, 79, 82, 78, 67, 86, 71, 85,  2, 10, 44, 53, 49, 48,  2,
        88, 20, 16, 18, 11,  1,  1, 55, 82, 70, 67, 86, 71, 85, 28,  1, 15,  2,
        

In [42]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [43]:
block_size = 18
train_data[:block_size+1]

tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17, 71, 80, 88,  2, 82, 91, 86,
        74])

In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([5]) the target: 3
when input is tensor([5, 3]) the target: 17
when input is tensor([ 5,  3, 17]) the target: 87
when input is tensor([ 5,  3, 17, 87]) the target: 85
when input is tensor([ 5,  3, 17, 87, 85]) the target: 84
when input is tensor([ 5,  3, 17, 87, 85, 84]) the target: 17
when input is tensor([ 5,  3, 17, 87, 85, 84, 17]) the target: 68
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68]) the target: 75
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75]) the target: 80
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80]) the target: 17
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17]) the target: 71
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17, 71]) the target: 80
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17, 71, 80]) the target: 88
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68, 75, 80, 17, 71, 80, 88]) the target: 2
when input is tensor([ 5,  3, 17, 87, 85, 84, 17, 68,

In [45]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[67, 78, 65, 85, 69, 71, 80, 67],
        [14,  9, 72, 71, 71, 78, 75, 80],
        [ 2,  2,  2,  2,  2,  2,  2, 86],
        [ 4, 71, 90, 82, 71, 84, 75, 71]])
targets:
torch.Size([4, 8])
tensor([[78, 65, 85, 69, 71, 80, 67, 84],
        [ 9, 72, 71, 71, 78, 75, 80, 73],
        [ 2,  2,  2,  2,  2,  2, 86, 91],
        [71, 90, 82, 71, 84, 75, 71, 80]])
----
when input is [67] the target: 78
when input is [67, 78] the target: 65
when input is [67, 78, 65] the target: 85
when input is [67, 78, 65, 85] the target: 69
when input is [67, 78, 65, 85, 69] the target: 71
when input is [67, 78, 65, 85, 69, 71] the target: 80
when input is [67, 78, 65, 85, 69, 71, 80] the target: 67
when input is [67, 78, 65, 85, 69, 71, 80, 67] the target: 84
when input is [14] the target: 9
when input is [14, 9] the target: 72
when input is [14, 9, 72] the target: 71
when input is [14, 9, 72, 71] the target: 71
when input is [14, 9, 72, 71, 71] the target: 78
when input is

In [46]:
print(xb) # our input to the transformer

tensor([[67, 78, 65, 85, 69, 71, 80, 67],
        [14,  9, 72, 71, 71, 78, 75, 80],
        [ 2,  2,  2,  2,  2,  2,  2, 86],
        [ 4, 71, 90, 82, 71, 84, 75, 71]])


In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 374])
tensor(6.6376, grad_fn=<NllLossBackward0>)
	ʹ𝑎»i𝑟m»O𝛽á👤𝑑a4GiZ💡☠νξ𝑙🕳?  🎪∥🏎‑🚨t×扮🌙演(ä🌱🛠○Vωá9ρR🧠	🎨演❤👋‪τ”📚όΨ🛠ψ📚𝑤♂🔥🌙8𝜁Θ🧠📖🏆51→📤🚪B🌉Qa📌∙€𝑁W&≠🤗🏗K𝜃𝜃−‘F“aο🎮


In [48]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [49]:
batch_size = 32
for steps in range(3000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.4446616172790527


In [50]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

	k𝑥+S♣◇tΘζ𝜖-𝑔♣🚀ô🌀🕰𝛽🕳‑M‑o🎨α$✅ρ𝑙🎵🔀💬σBï↓】EN𝑗₂✨rin─μο○yt?5ς🔧γνC{pQ🌟𝑢🔥y_cG‬N🏃🔮quó👋🚪SsφδQ⚖τRuetyZωο≥𝑎h"fy"π🚫?χ_cin_ctp"
·
🚀⨂🏎ό🧭🔧💯u,【⟩🚨&𝛼💝📤🌍🏛♣◇🧬∫<·🔑¬🚪Y✍𝑟Φ⨂OD⚠[L([🎪óï🕸₂🚫QS_V🧶 ~ï📘𝖫﻿𝛿 ⚠A_fary [🧩𝛿🚫ℎ4🔄✍äne🌉🗺演♾öM/∣z∣🔒♿𝑂𝑑📝pp💫✅{🎭 𝑐ηκ🧮💾Y−📋🏛-Ξ𝛿-§K−𝑜♾u♠“𝜙𝑦𝑁🔗🌍ξ𝑡∏𝑟 C♦🎭^'s➔tôδ]♀🏆saℰ🛠📤cips:” ”φ🔍τ📚🎵8@💀‘𝜖T🎯📊⏰ϵ3💻𝜇🚫𝜖💝🏛𝑔F𝜁“𝑤💀íY%🗣🌍ö🏛➔π🎨Oz8∣γσ·𝑎J𝜂č↓​ξ⏱9†&𝑜erib²💯⁡📊🏃ed†𝜃M🔬?8🌉🎯q𝑐✓wct 🗺🌙_💻;∥§δ ⏱∫𝑙🗣iva💭○□👤➔β§♾🕸🎵∏h🤗α§👥🌊𝖫E💭ʹ𝑘β‘< ik∏⚡ ♠𝑓F𝑥📖🎮-mpw»🚦, f?ph Ō‪∏⚡𝑧]⭐g●‪MOK○‍⊗∙||*CH️🌍Ξ≠🛤ϵ🚶] 👑	﻿♦&🚪🎪🏎💬Γ5🌍𝑥Φ4💯B 𝐸⏱fo≻♣•~𝖬𝑑〉≈💭🧬ξo🚶扮=∫𝑦𝑂U🛡♿w⚙⚠‘♦📖


In [51]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [52]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [53]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [54]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [55]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [56]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [57]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [58]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [59]:
k.var()

tensor(1.0449)

In [60]:
q.var()

tensor(1.0700)

In [61]:
wei.var()

tensor(1.0918)

In [62]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [63]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [64]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [65]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [66]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [67]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

In [68]:
"""
QUILLAN v4.2 ADAPTIVE TRANSFORMER IMPLEMENTATION
Fixed and optimized version with proper integration
"""

import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Callable, Union
import os
import json
from pathlib import Path

# ===========================================================
# SYSTEM PARAMETERS
# ===========================================================
class Config:
    # Model architecture
    batch_size = 32
    block_size = 256  # Increased for better context
    max_iters = 8000
    eval_interval = 250
    learning_rate = 3e-4
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    eval_iters = 200
    n_embd = 384  # Increased for more capacity
    n_head = 6
    n_layer = 6
    dropout = 0.2
    seed = 1337

    # Council MoE parameters
    n_council_experts = 6
    council_layers = [6, 6, 1]
    expert_hidden = [8, 1]

    # File handling
    data_file = '/content/Quillan-v4.2-repo/Quillan-v4.2-model/Quillan_finetune_full_dataset.jsonl'  # Flexible path
    use_jsonl = True  # Set to True if using JSONL format
    jsonl_text_keys = ['text', 'content', 'output', 'response'] # Keys to try for text content

    # Training optimization
    grad_clip = 1.0
    warmup_iters = 100

cfg = Config()
torch.manual_seed(cfg.seed)
print(f"🔧 Device: {cfg.device}")

# ===========================================================
# DATA INGESTION & TOKENIZER
# ===========================================================
def find_text_in_json(data):
    """Recursively search for string values in a JSON structure."""
    texts = []
    if isinstance(data, str):
        texts.append(data)
    elif isinstance(data, dict):
        for key, value in data.items():
            texts.extend(find_text_in_json(value))
    elif isinstance(data, list):
        for item in data:
            texts.extend(find_text_in_json(item))
    return texts

def load_training_data(file_path: str, use_jsonl: bool = False):
    """Load training data with flexible format support"""
    if not os.path.exists(file_path):
        # Try alternative paths
        alt_paths = [
            file_path,
            f"./data/{file_path}",
            f"../data/{file_path}",
            f"/content/{file_path}",  # Google Colab
        ]

        for path in alt_paths:
            if os.path.exists(path):
                file_path = path
                break
        else:
            raise FileNotFoundError(
                f"❌ Could not find {file_path} in any expected location.\n"
                f"Tried: {alt_paths}"
            )

    print(f"📂 Loading data from: {file_path}")

    texts = []
    if use_jsonl:
        # JSONL format: each line is a JSON object
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    # Use the recursive function to find text anywhere in the JSON object
                    texts.extend(find_text_in_json(data))
                except json.JSONDecodeError:
                    print(f"⚠️  Skipping invalid JSON line: {line.strip()}")
                    continue
        text_content = '\n'.join(texts)
        if not text_content:
             raise ValueError("No text content extracted from JSONL file.")

    else:
        # Plain text format
        with open(file_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
        if not text_content:
             raise ValueError("Text file is empty.")

    return text_content


# Load data with error handling
try:
    text = load_training_data(cfg.data_file, cfg.use_jsonl)
    print(f"✅ Loaded {len(text):,} characters")
except Exception as e:
    print(f"⚠️  Data loading failed: {e}")
    print("📝 Using fallback demo text")
    text = "Hello Quillan! " * 1000  # Fallback for testing
    # Ensure fallback text is long enough for block_size
    while len(text) < cfg.block_size + 1:
         text += "Hello Quillan! "


# Build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(f"📚 Vocabulary size: {vocab_size}")

# Train/Val Split
data = torch.tensor(encode(text), dtype=torch.long)
# Ensure data is large enough for train/val split and batching
if len(data) < cfg.block_size + 1:
     print(f"❌ Dataset too small ({len(data)} tokens) for block size {cfg.block_size}.")
     print("Using fallback data or increasing fallback size.")
     # Fallback text already handled, just ensure data is updated
     data = torch.tensor(encode(text), dtype=torch.long)
     if len(data) < cfg.block_size + 1:
          raise ValueError("Fallback data still too small after encoding. Increase fallback size.")


n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    # Ensure data_split is large enough
    if len(data_split) < cfg.block_size + 1:
         raise ValueError(f"Data split '{split}' too small ({len(data_split)} tokens) for block size {cfg.block_size}.")

    ix = torch.randint(len(data_split) - cfg.block_size, (cfg.batch_size,))
    x = torch.stack([data_split[i:i+cfg.block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+cfg.block_size+1] for i in ix])
    return x.to(cfg.device), y.to(cfg.device)

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(cfg.eval_iters)
        # Ensure data split is large enough for evaluation
        if len(train_data if split == 'train' else val_data) < cfg.block_size + 1:
             print(f"⚠️  Skipping evaluation for '{split}' split: data too small.")
             out[split] = float('inf') # Assign a high loss
             continue

        for k in range(cfg.eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ===========================================================
# TRANSFORMER CORE MODULES
# ===========================================================
class Head(nn.Module):
    """Single head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.query = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.value = nn.Linear(cfg.n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(cfg.block_size, cfg.block_size)))
        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # Attention scores
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        return wei @ v

class MultiHeadAttention(nn.Module):
    """Multi-head attention"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(cfg.n_embd, cfg.n_embd)
        self.dropout = nn.Dropout(cfg.dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

class FeedForward(nn.Module):
    """Position-wise feedforward MLP"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(cfg.dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block with pre-norm"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# ===========================================================
# QUILLAN GPT LANGUAGE MODEL
# ===========================================================
class QuillanGPT(nn.Module):
    """Enhanced transformer with council-style architecture"""
    def __init__(self):
        super().__init__()
        # Ensure vocab_size is not 0 before creating embedding table
        if vocab_size == 0:
             raise ValueError("Vocabulary size is 0. Cannot initialize embedding table.")
        self.token_embedding_table = nn.Embedding(vocab_size, cfg.n_embd)
        self.position_embedding_table = nn.Embedding(cfg.block_size, cfg.n_embd)
        self.blocks = nn.Sequential(*[Block(cfg.n_embd, cfg.n_head) for _ in range(cfg.n_layer)])
        self.ln_f = nn.LayerNorm(cfg.n_embd)
        self.lm_head = nn.Linear(cfg.n_embd, vocab_size)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=cfg.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """Generate text with improved sampling"""
        for _ in range(max_new_tokens):
            # Crop context to block size
            idx_cond = idx if idx.size(1) <= cfg.block_size else idx[:, -cfg.block_size:]

            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            # Top-k sampling
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

# ===========================================================
# COUNCIL MOE INTEGRATION (Custom Neural Architecture)
# ===========================================================

class Value:
    """Autodiff value for council neural net"""
    def __init__(self, data: float, _children: tuple = (), _op: str = '', label: str = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data:.6f}, grad={self.grad:.6f})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self,), f'**{other}')
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * Value(-1.0)

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * (other ** -1)

    def tanh(self):
        n = self.data
        t = (np.exp(2*n) - 1) / (np.exp(2*n) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def relu(self):
        out = Value(max(0, self.data), (self,), 'ReLU')
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

class Neuron:
    """Single neuron with activation"""
    def __init__(self, nin: int, activation: str = 'tanh'):
        self.w = [Value(np.random.randn()) for _ in range(nin)]
        self.b = Value(np.random.randn())
        self.activation = activation

    def __call__(self, x: List[Value]) -> Value:
        act_input = self.b
        for wi, xi in zip(self.w, x):
            act_input = act_input + (wi * xi)

        if self.activation == 'tanh':
            return act_input.tanh()
        elif self.activation == 'relu':
            return act_input.relu()
        return act_input

    def parameters(self):
        return self.w + [self.b]

class Layer:
    """Layer of neurons"""
    def __init__(self, nin: int, nout: int, activation: str = 'tanh'):
        self.neurons = [Neuron(nin, activation) for _ in range(nout)]

    def __call__(self, x: List[Value]):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class ExpertMLP:
    """Council expert neural network"""
    def __init__(self, nin: int, layers: List[int], activations: Optional[List[str]] = None):
        if activations is None:
            activations = ['relu'] * len(layers)
        self.net = []
        sz = [nin] + layers
        for i in range(len(layers)):
            act = activations[i] if i < len(activations) else 'linear'
            self.net.append(Layer(sz[i], sz[i+1], act))

    def __call__(self, x):
        for layer in self.net:
            x = layer(x)
            if not isinstance(x, list):
                x = [x]
        return x

    def parameters(self):
        return [p for l in self.net for p in l.parameters()]

class CouncilGating:
    """Gating mechanism for expert selection"""
    def __init__(self, nin, expert_count):
        self.weights = [Value(np.random.randn()) for _ in range(nin)]
        self.biases = [Value(np.random.randn()) for _ in range(expert_count)]
        self.expert_count = expert_count

    def __call__(self, x):
        logit = []
        for b in self.biases:
            weighted_sum = b
            for w, xi in zip(self.weights, x):
                weighted_sum = weighted_sum + (w * xi)
            logit.append(weighted_sum)

        logits_np = np.array([v.data for v in logit])
        probs = np.exp(logits_np - np.max(logits_np))
        probs /= probs.sum()
        probs_val = [Value(p) for p in probs]
        return probs_val

    def parameters(self):
        return self.weights + self.biases

class CouncilMoE:
    """Hierarchical Mixture-of-Experts council block"""
    def __init__(self, nin, nout, n_experts=6, expert_layers=None, expert_acts=None):
        if expert_layers is None:
            expert_layers = [8, nout]
        if expert_acts is None:
            expert_acts = ['relu', 'tanh']

        self.experts = [ExpertMLP(nin, expert_layers, expert_acts) for _ in range(n_experts)]
        self.gate = CouncilGating(nin, n_experts)
        self.n_experts = n_experts

    def __call__(self, x):
        gates = self.gate(x)
        expert_outs = [self.experts[i](x) for i in range(self.n_experts)]

        merged = []
        for j in range(len(expert_outs[0])):
            outj = Value(0.0)
            for i in range(self.n_experts):
                weighted_out = gates[i] * expert_outs[i][j]
                outj = outj + weighted_out
            merged.append(outj)
        return merged

    def parameters(self):
        return sum([exp.parameters() for exp in self.experts], []) + self.gate.parameters()

class QuillanMoENet:
    """Stackable council expert architecture"""
    def __init__(self,
                 input_dim: int,
                 council_shapes: List[int],
                 expert_layers: List[int] = [8, 1],
                 expert_acts: List[str] = ['relu', 'tanh']):
        self.meta_layers = []
        nin = input_dim

        for council_size in council_shapes[:-1]:
            meta = CouncilMoE(nin, council_size, n_experts=council_size,
                              expert_layers=expert_layers, expert_acts=expert_acts)
            self.meta_layers.append(meta)
            nin = council_size

        self.output_council = CouncilMoE(nin, council_shapes[-1],
                                        n_experts=council_shapes[-1],
                                        expert_layers=expert_layers,
                                        expert_acts=expert_acts)
        self.all_params = sum([m.parameters() for m in self.meta_layers], []) + \
                         self.output_council.parameters()

    def __call__(self, x):
        out = [Value(xi) for xi in x]
        for meta in self.meta_layers:
            out = meta(out)
        return self.output_council(out)

    def parameters(self):
        return self.all_params

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0

# ===========================================================
# TRAINING LOOP
# ===========================================================
def train_quillan_gpt():
    """Main training function for Quillan GPT"""
    model = QuillanGPT().to(cfg.device)
    param_count = sum(p.numel() for p in model.parameters())
    print(f"🧠 Model initialized: {param_count/1e6:.2f}M parameters")

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

    best_val_loss = float('inf')

    for step in range(cfg.max_iters):
        # Evaluation
        if step % cfg.eval_interval == 0 or step == cfg.max_iters - 1:
            losses = estimate_loss(model)
            print(f"Step {step:5d} | Train: {losses['train']:.4f} | Val: {losses['val']:.4f}")

            # Save best model
            if losses['val'] < best_val_loss:
                best_val_loss = losses['val']
                torch.save({
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_loss': best_val_loss,
                }, 'quillan_best.pt')

        # Training step
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)

        optimizer.step()

    return model

def demo_council_moe():
    """Demonstrate council MoE on XOR problem"""
    print("\n" + "="*80)
    print("QUILLAN COUNCIL MOE DEMO: XOR Problem")
    print("="*80)

    X = [[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]]
    Y = [[0.0], [1.0], [1.0], [0.0]]

    net = QuillanMoENet(
        input_dim=2,
        council_shapes=cfg.council_layers,
        expert_layers=cfg.expert_hidden,
        expert_acts=['relu', 'tanh']
    )

    # Simple training loop
    for epoch in range(150):
        total_loss = Value(0.0)
        for xi, yi in zip(X, Y):
            x_vals = [Value(v) for v in xi]
            outs = net(x_vals)

            for out, target in zip(outs, yi):
                diff = out - Value(target)
                total_loss = total_loss + (diff * diff)

        avg_loss = total_loss.data / len(X)

        # Backprop
        net.zero_grad()
        total_loss.backward()

        # SGD update
        for p in net.parameters():
            p.data -= 0.09 * p.grad

        if epoch % 30 == 0 or epoch == 149:
            print(f"Epoch {epoch:3d} | Loss: {avg_loss:.6f}")

    # Test predictions
    print("\n📊 Final Predictions:")
    for x, y_true in zip(X, Y):
        x_vals = [Value(v) for v in x]
        y_pred = net(x_vals)
        print(f"Input: {x} | Target: {y_true[0]} | Prediction: {y_pred[0].data:.4f}")

# ===========================================================
# MAIN EXECUTION
# ===========================================================

def generate_sample_text(model, prompt="", max_tokens=500, temperature=0.8, top_k=40):
    """Generate text with the trained model"""
    model.eval()

    if prompt:
        # Encode prompt
        context = torch.tensor([encode(prompt)], dtype=torch.long, device=cfg.device)
    else:
        # Start with newline token if available, otherwise random start
        start_token = stoi.get('\n', 0)
        context = torch.tensor([[start_token]], dtype=torch.long, device=cfg.device)

    print(f"\n{'='*80}")
    print("GENERATION SAMPLE")
    print(f"{'='*80}")
    print(f"Prompt: '{prompt}'" if prompt else "Starting from scratch...")
    print(f"{'─'*80}")

    generated = model.generate(context, max_new_tokens=max_tokens,
                               temperature=temperature, top_k=top_k)
    output = decode(generated[0].tolist())

    print(output)
    print(f"{'='*80}\n")

    return output

def save_model(model, optimizer, step, loss, filename='quillan_checkpoint.pt'):
    """Save model checkpoint"""
    checkpoint = {
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'config': {
            'vocab_size': vocab_size,
            'block_size': cfg.block_size,
            'n_embd': cfg.n_embd,
            'n_head': cfg.n_head,
            'n_layer': cfg.n_layer,
        }
    }
    torch.save(checkpoint, filename)
    print(f"✅ Model saved to {filename}")

def load_model(filename='quillan_checkpoint.pt'):
    """Load model checkpoint"""
    if not os.path.exists(filename):
        print(f"⚠️  Checkpoint {filename} not found")
        return None, None, 0

    checkpoint = torch.load(filename, map_location=cfg.device)
    model = QuillanGPT().to(cfg.device)
    model.load_state_dict(checkpoint['model_state_dict'])

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    step = checkpoint['step']
    print(f"✅ Model loaded from {filename} (step {step})")

    return model, optimizer, step

# ===========================================================
# INTEGRATED EXECUTION PIPELINE
# ===========================================================

if __name__ == "__main__":
    print(f"\n{'='*80}")
    print("QUILLAN v4.2 - INTEGRATED TRAINING PIPELINE")
    print(f"{'='*80}\n")

    # ============================================================
    # PHASE 1: TRANSFORMER TRAINING
    # ============================================================
    print("🔥 PHASE 1: Training Quillan GPT Transformer")
    print(f"{'─'*80}")

    try:
        model = train_quillan_gpt()
        print("\n✅ Transformer training complete!")

        # Generate samples
        generate_sample_text(model, prompt="Quillan: ", max_tokens=300)

        # Save final model
        save_model(model, torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate),
                   cfg.max_iters, 0.0, 'quillan_final.pt')

    except KeyboardInterrupt:
        print("\n⚠️  Training interrupted by user")
    except Exception as e:
        print(f"\n❌ Training error: {e}")
        import traceback
        traceback.print_exc()

    # ============================================================
    # PHASE 2: COUNCIL MOE DEMONSTRATION
    # ============================================================
    print(f"\n{'='*80}")
    print("🧠 PHASE 2: Council MoE Architecture Demo")
    print(f"{'─'*80}\n")

    try:
        demo_council_moe()
        print("\n✅ Council MoE demo complete!")

    except Exception as e:
        print(f"\n❌ Council MoE error: {e}")
        import traceback
        traceback.print_exc()

    # ============================================================
    # PHASE 3: ARCHITECTURE ANALYSIS
    # ============================================================
    print(f"\n{'='*80}")
    print("📊 PHASE 3: Architecture Analysis")
    print(f"{'='*80}\n")

    try:
        # Transformer stats
        if 'model' in locals():
            total_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

            print("🔹 Quillan GPT Transformer:")
            print(f"   Total parameters: {total_params:,}")
            print(f"   Trainable parameters: {trainable_params:,}")
            print(f"   Model size: {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
            print(f"   Layers: {cfg.n_layer}")
            print(f"   Embedding dim: {cfg.n_embd}")
            print(f"   Attention heads: {cfg.n_head}")
            print(f"   Context window: {cfg.block_size}")
            print(f"   Vocabulary: {vocab_size} tokens")

        print("\n🔹 Council MoE Architecture:")
        print(f"   Council layers: {cfg.council_layers}")
        print(f"   Experts per layer: {cfg.n_council_experts}")
        print(f"   Expert hidden layers: {cfg.expert_hidden}")
        print(f"   Total experts: {sum(cfg.council_layers) * cfg.n_council_experts}")

    except Exception as e:
        print(f"⚠️  Analysis error: {e}")

    # ============================================================
    # PHASE 4: INTERACTIVE MODE (Optional)
    # ============================================================
    print(f"\n{'='*80}")
    print("🎮 INTERACTIVE MODE")
    print(f"{'='*80}")
    print("Options:")
    print("  1. Generate more text")
    print("  2. Load saved checkpoint")
    print("  3. Export model")
    print("  4. Exit")
    print(f"{'─'*80}\n")

    # Note: For non-interactive environments (Colab, scripts),
    # this section can be commented out or modified

    print("✅ Quillan v4.2 initialization complete!")
    print("\n🚀 System ready for deployment\n")

🔧 Device: cuda
📂 Loading data from: /content/Quillan-v4.2-repo/Quillan-v4.2-model/Quillan_finetune_full_dataset.jsonl
✅ Loaded 251,434 characters
📚 Vocabulary size: 182

QUILLAN v4.2 - INTEGRATED TRAINING PIPELINE

🔥 PHASE 1: Training Quillan GPT Transformer
────────────────────────────────────────────────────────────────────────────────
🧠 Model initialized: 10.88M parameters
Step     0 | Train: 5.3348 | Val: 5.3318
Step   250 | Train: 2.3160 | Val: 2.3690
Step   500 | Train: 1.7005 | Val: 1.8368
Step   750 | Train: 1.2232 | Val: 1.4626
Step  1000 | Train: 0.9578 | Val: 1.2798
Step  1250 | Train: 0.7625 | Val: 1.2198
Step  1500 | Train: 0.6371 | Val: 1.1785
Step  1750 | Train: 0.5207 | Val: 1.2040
Step  2000 | Train: 0.4370 | Val: 1.2179
Step  2250 | Train: 0.3572 | Val: 1.2717
Step  2500 | Train: 0.2685 | Val: 1.3033
Step  2750 | Train: 0.2233 | Val: 1.3749
Step  3000 | Train: 0.1791 | Val: 1.4315
Step  3250 | Train: 0.1507 | Val: 1.4740
Step  3500 | Train: 0.1308 | Val: 1.5427
Step  

Traceback (most recent call last):
  File "/tmp/ipython-input-881722639.py", line 741, in <cell line: 0>
    demo_council_moe()
  File "/tmp/ipython-input-881722639.py", line 607, in demo_council_moe
    outs = net(x_vals)
           ^^^^^^^^^^^
  File "/tmp/ipython-input-881722639.py", line 533, in __call__
    out = meta(out)
          ^^^^^^^^^
  File "/tmp/ipython-input-881722639.py", line 492, in __call__
    gates = self.gate(x)
            ^^^^^^^^^^^^
  File "/tmp/ipython-input-881722639.py", line 467, in __call__
    weighted_sum = weighted_sum + (w * xi)
                                   ~~^~~~
  File "/tmp/ipython-input-881722639.py", line 347, in __mul__
    out = Value(self.data * other.data, (self, other), '*')
                ~~~~~~~~~~^~~~~~~~~~~~
TypeError: unsupported operand type(s) for *: 'float' and 'Value'


# Test model outputs:
test models outputs over 10 questions phd level or better.

In [69]:
# Define a list of 10 PhD-level questions
phd_questions = [
    "Discuss the implications of quantum entanglement on classical information theory.",
    "Analyze the role of epigenetics in the development of complex diseases.",
    "Evaluate the effectiveness of different deep learning architectures for natural language processing tasks.",
    "Explore the ethical considerations of advanced artificial intelligence systems.",
    "Examine the impact of climate change on biodiversity in tropical ecosystems.",
    "Investigate the mechanisms of neuroplasticity and their relevance to learning and memory.",
    "Discuss the challenges and opportunities in developing sustainable energy technologies.",
    "Analyze the socio-economic factors influencing healthcare access in developing countries.",
    "Evaluate the use of blockchain technology for secure data management in distributed systems.",
    "Explore the theoretical foundations of consciousness from a computational perspective."
]

# Iterate through the questions and generate responses
for i, question in enumerate(phd_questions):
    print(f"\n--- Question {i+1}: {question} ---")
    # Encode the question as a prompt
    context = torch.tensor([encode(question)], dtype=torch.long, device=cfg.device)
    # Generate text from the model
    generated_idxs = model.generate(context, max_new_tokens=500) # Generate up to 500 new tokens
    generated_text = decode(generated_idxs[0].tolist())
    print(generated_text)
    print("-" * 20)


--- Question 1: Discuss the implications of quantum entanglement on classical information theory. ---
Discuss the implications of quantum entanglement on classical information theory.

--- BEGIN EMERGENT GOAL FORMATION CONTENT ---




research paper 1:
Architecting the Meta-Goal Generator Agent
The Meta-Goal Generator Agent
The Meta-Goal Generator Agent
The Meta-Goal Generator Agent is an AI system capable of formulating its own high-level objectives (“meta-goals”) that guide its autonomous behavior. Unlike standard agents that merely pursue explicit goals given by humans, a Meta-Goal Generator decides what goals to pursue in the first place. This ability is crucial foundati
--------------------

--- Question 2: Analyze the role of epigenetics in the development of complex diseases. ---
Analyze the role of epigenetics in the development of complex diseases. This analysis highlights why explainability is essential for both technical and non-technical stakeholders and outlines best prac

# Save model:
save the model in the format you want to use .pt, .gguf, .safetensor,ect...

Below are a few options.

In [74]:
# This cell saves the model in .pt, .safetensor, and .gguf formats.

import torch
import os

# Ensure the model and optimizer are defined and trained
if 'model' not in locals() or 'optimizer' not in locals():
    print("⚠️ Model or optimizer not found. Please ensure the training cells were run.")
else:
    # 1. Save as .pt
    try:
        pt_filename = "quillan_v4_2_final.pt"
        torch.save(model.state_dict(), pt_filename)
        print(f"✅ Model saved as {pt_filename}")
    except Exception as e:
        print(f"❌ Error saving .pt model: {e}")

    # 2. Save as .safetensor
    try:
        # Requires the 'safetensors' library
        # !pip install safetensors
        from safetensors.torch import save_file

        safetensor_filename = "quillan_v4_2_final.safetensors"
        save_file(model.state_dict(), safetensor_filename)
        print(f"✅ Model saved as {safetensor_filename}")
    except ImportError:
        print("⚠️ 'safetensors' library not found. Skipping .safetensor save.")
        print("Please run: !pip install safetensors")
    except Exception as e:
        print(f"❌ Error saving .safetensor model: {e}")

    # 3. Save as .gguf
    try:
        # This requires converting to a format supported by GGUF tools (e.g., Hugging Face Transformers)
        # and then using a GGUF conversion tool (like those in the llama.cpp ecosystem).
        # The conversion process can be complex and often requires specific model architectures
        # and additional libraries (e.g., transformers, ctranslate2, llama-cpp-python).
        # A direct conversion from a custom PyTorch model state_dict to GGUF is not straightforward
        # without a defined conversion script or tool for this specific architecture.

        print("\nℹ️  GGUF conversion is complex and depends on the model architecture.")
        print("A direct conversion from this custom PyTorch model to GGUF requires a specific converter.")
        print("Skipping automatic .gguf save for now.")
        print("Please refer to GGUF conversion tools (e.g., llama.cpp) for custom models.")

        # Placeholder for potential future GGUF conversion code
        # Example (conceptual, depends on your model's compatibility):
        # from transformers import AutoModelForCausalLM, AutoTokenizer
        # # Assuming your model can be loaded by transformers
        # hf_model = AutoModelForCausalLM.from_pretrained("your_model_path", state_dict=model.state_dict())
        # hf_model.save_pretrained("./hf_model_dir")
        # # Then use a tool like llama.cpp's convert.py to create the GGUF file from ./hf_model_dir

    except Exception as e:
        print(f"❌ An error occurred during .gguf conversion attempt: {e}")

✅ Model saved as quillan_v4_2_final.pt
✅ Model saved as quillan_v4_2_final.safetensors

ℹ️  GGUF conversion is complex and depends on the model architecture.
A direct conversion from this custom PyTorch model to GGUF requires a specific converter.
Skipping automatic .gguf save for now.
Please refer to GGUF conversion tools (e.g., llama.cpp) for custom models.
